In [43]:
# import other .py file to .ipynb file
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.abspath(''))))
sys.path

['/usr/lib/python311.zip',
 '/usr/lib/python3.11',
 '/usr/lib/python3.11/lib-dynload',
 '',
 '/home/user/pandas-polar-dask/.venv/lib/python3.11/site-packages',
 '/home/user/pandas-polar-dask',
 '/home/user/pandas-polar-dask']

In [44]:
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from dask.delayed import delayed
# dask's just effective with large file (>= 1G), if not, use pandas is better

In [45]:
file_name = '/home/user/pandas-polar-dask/data/_1_bronze data/sales_data_sample.xlsx'

In [46]:
# Read the Excel file lazily using delayed
@delayed
def read_excel_lazy(file_path: str = '', sheet_name: str='') -> pd.DataFrame :
    return pd.read_excel(file_path, sheet_name= sheet_name)

def to_dask_df(file_name: str = '', sheet_name: str= '') -> dd.DataFrame:
    # Create delayed pandas DataFrame
    parts = [read_excel_lazy(file_name, sheet_name)]
    # Convert to Dask DataFrame
    df = dd.from_delayed(parts)
    return df



## Sales_Data

In [47]:
# Trigger computation and display the head
sales_data = to_dask_df(file_name, 'Sales_Data')
sales_data.head()
# lazy frame need .compute() to trigger computation

,Order_ID,Date,Category,Region,Customer_Segment,Payment_Method,Quantity,Unit_Price,Discount_Rate,Customer_Rating,Returned,Subcategory,Total_Price,Discount_Amount,Net_Sales,Shipping_Cost,Profit_Margin,Profit,Month
0,ORD-10000,1970-01-01 07:30:45,Books,East,Business,PayPal,1,316.58,0.20,5.0,False,Comics,316.58,63.3160,253.2640,0.0,0.33,83.577120,2024-07
1,ORD-10001,1970-01-01 07:30:46,Sports,West,Premium,Bank Transfer,5,279.76,0.10,NaN,True,Water Sports,1398.80,139.8800,1258.9200,0.0,0.20,251.784000,2025-03
2,ORD-10002,1970-01-01 07:30:46,Home & Kitchen,Central,Business,Bank Transfer,6,209.61,0.05,4.0,False,Furniture,1257.66,62.8830,1194.7770,0.0,0.37,442.067490,2024-12
3,ORD-10003,1970-01-01 07:30:45,Home & Kitchen,North,Regular,Credit Card,3,265.47,0.15,4.0,False,Kitchenware,796.41,119.4615,676.9485,0.0,0.23,155.698155,2024-07
4,ORD-10004,1970-01-01 07:30:45,Electronics,North,New,Bank Transfer,9,449.32,0.00,4.0,False,Accessories,4043.88,0.0000,4043.8800,0.0,0.37,1496.235600,2024-06


In [48]:
sales_data.describe().compute()

,Date,Quantity,Unit_Price,Discount_Rate,Customer_Rating,Total_Price,Discount_Amount,Net_Sales,Shipping_Cost,Profit_Margin,Profit
count,1000,1000.000000,1000.000000,949.000000,945.000000,1000.000000,1000.000000,1000.000000,958.000000,1000.000000,1000.000000
min,1970-01-01 07:30:45,1.000000,10.110000,0.000000,1.000000,19.870000,0.000000,18.876500,0.000000,0.200000,4.551240
25%,1970-01-01 07:30:45,3.000000,135.702500,0.050000,3.000000,422.905000,16.000000,376.503250,0.000000,0.250000,109.791080
50%,1970-01-01 07:30:46,5.000000,246.845000,0.100000,4.000000,1046.475000,72.346000,924.352500,0.000000,0.300000,283.817680
75%,1970-01-01 07:30:46,7.000000,378.147500,0.150000,5.000000,1936.402500,182.017500,1734.259000,0.000000,0.350000,500.323500
max,1970-01-01 07:30:46,9.000000,499.410000,0.200000,5.000000,4474.980000,860.022000,4304.430000,19.860000,0.400000,1496.235600
mean,NaN,5.078000,254.370130,0.098367,3.708995,1305.129780,129.438433,1175.691347,1.433946,0.298400,351.559802
std,NaN,2.533838,140.757989,0.069393,1.093593,1048.253701,155.924665,951.982940,4.238441,0.057922,298.512945


In [49]:
sales_data.info()

<class 'dask.dataframe.dask_expr.DataFrame'>
Columns: 19 entries, Order_ID to Month
dtypes: datetime64[ns](1), bool(1), float64(9), int64(1), string(7)

In [50]:
sales_data.isna().sum().compute()

Order_ID             0
Date                 0
Category             0
Region               0
Customer_Segment     0
Payment_Method       0
Quantity             0
Unit_Price           0
Discount_Rate       51
Customer_Rating     55
Returned             0
Subcategory          0
Total_Price          0
Discount_Amount      0
Net_Sales            0
Shipping_Cost       42
Profit_Margin        0
Profit               0
Month                0
dtype: int64

In [51]:
sales_data.count().compute() # not contain nan null

Order_ID            1000
Date                1000
Category            1000
Region              1000
Customer_Segment    1000
Payment_Method      1000
Quantity            1000
Unit_Price          1000
Discount_Rate        949
Customer_Rating      945
Returned            1000
Subcategory         1000
Total_Price         1000
Discount_Amount     1000
Net_Sales           1000
Shipping_Cost        958
Profit_Margin       1000
Profit              1000
Month               1000
dtype: int64

In [52]:
sales_data.shape[0]

<dask_expr.expr.Scalar: expr=ArrowStringConversion(frame=FromDelayed(0dd7de7)).size() // 19, dtype=int64>

In [53]:
sales_data.columns

Index(['Order_ID', 'Date', 'Category', 'Region', 'Customer_Segment',
       'Payment_Method', 'Quantity', 'Unit_Price', 'Discount_Rate',
       'Customer_Rating', 'Returned', 'Subcategory', 'Total_Price',
       'Discount_Amount', 'Net_Sales', 'Shipping_Cost', 'Profit_Margin',
       'Profit', 'Month'],
      dtype='object')

In [54]:
len(sales_data)

1000

In [55]:
# count data per columns
half_len_sales_data = len(sales_data)/2
for col in sales_data.columns:
    value_count_per_col= sales_data[col].value_counts().compute()
    if len(value_count_per_col) < half_len_sales_data:
        print(value_count_per_col)

Date
1970-01-01 07:30:45    311
1970-01-01 07:30:46    689
Name: count, dtype: int64
Category
Books             186
Clothing          204
Electronics       208
Home & Kitchen    195
Sports            207
Name: count, dtype: int64[pyarrow]
Region
Central    201
East       184
North      192
South      192
West       231
Name: count, dtype: int64[pyarrow]
Customer_Segment
Business    245
New         273
Premium     248
Regular     234
Name: count, dtype: int64[pyarrow]
Payment_Method
Bank Transfer       187
Cash on Delivery    203
Credit Card         201
Debit Card          192
PayPal              217
Name: count, dtype: int64[pyarrow]
Quantity
1    103
2    100
3    112
4    115
5    100
6    134
7    121
8    107
9    108
Name: count, dtype: int64
Discount_Rate
0.00    172
0.05    227
0.10    196
0.15    168
0.20    186
Name: count, dtype: int64
Customer_Rating
1.0     44
2.0     95
3.0    193
4.0    373
5.0    240
Name: count, dtype: int64
Returned
False    889
True     111
Name: coun

In [56]:

for col in sales_data.columns:
    nunique_per_col= sales_data[col].nunique().compute() #Return number of unique elements
    if nunique_per_col < len(sales_data):
        print(col, nunique_per_col)

Date 2
Category 5
Region 5
Customer_Segment 4
Payment_Method 5
Quantity 9
Unit_Price 994
Discount_Rate 5
Customer_Rating 5
Returned 2
Subcategory 23
Total_Price 998
Discount_Amount 817
Net_Sales 999
Shipping_Cost 104
Profit_Margin 21
Profit 999
Month 13


In [86]:
sales_data_clean = sales_data.copy()

In [87]:
sales_data_clean['Customer_Rating'] = sales_data_clean['Customer_Rating'].fillna(sales_data_clean['Customer_Rating'].median())

In [88]:
sales_data_clean['Shipping_Cost'] = sales_data_clean['Shipping_Cost'].fillna(sales_data_clean['Shipping_Cost'].median())

In [89]:
sales_data_clean['Month'] = dd.to_datetime(sales_data_clean['Month'], format='%Y-%m')
sales_data_clean['Month'].head()

0   2024-07-01
1   2025-03-01
2   2024-12-01
3   2024-07-01
4   2024-06-01
Name: Month, dtype: datetime64[ns]

In [90]:
sales_data_clean['Day'] = sales_data_clean['Month'].dt.day
sales_data_clean['Only_Month'] = sales_data_clean['Month'].dt.month
sales_data_clean['Year'] = sales_data_clean['Month'].dt.year
sales_data_clean['DayOfWeek'] = sales_data_clean['Month'].dt.dayofweek
sales_data_clean.compute().head()

,Order_ID,Date,Category,Region,Customer_Segment,Payment_Method,Quantity,Unit_Price,Discount_Rate,Customer_Rating,...,Discount_Amount,Net_Sales,Shipping_Cost,Profit_Margin,Profit,Month,Day,Only_Month,Year,DayOfWeek
0,ORD-10000,1970-01-01 07:30:45,Books,East,Business,PayPal,1,316.58,0.20,5.0,...,63.3160,253.2640,0.0,0.33,83.577120,2024-07-01,1,7,2024,0
1,ORD-10001,1970-01-01 07:30:46,Sports,West,Premium,Bank Transfer,5,279.76,0.10,4.0,...,139.8800,1258.9200,0.0,0.20,251.784000,2025-03-01,1,3,2025,5
2,ORD-10002,1970-01-01 07:30:46,Home & Kitchen,Central,Business,Bank Transfer,6,209.61,0.05,4.0,...,62.8830,1194.7770,0.0,0.37,442.067490,2024-12-01,1,12,2024,6
3,ORD-10003,1970-01-01 07:30:45,Home & Kitchen,North,Regular,Credit Card,3,265.47,0.15,4.0,...,119.4615,676.9485,0.0,0.23,155.698155,2024-07-01,1,7,2024,0
4,ORD-10004,1970-01-01 07:30:45,Electronics,North,New,Bank Transfer,9,449.32,0.00,4.0,...,0.0000,4043.8800,0.0,0.37,1496.235600,2024-06-01,1,6,2024,5


In [62]:
# sales_data_clean = sales_data_clean.rename(columns={'Month': 'Order_Month'})
# sales_data_clean.compute().head()
# rename column

In [91]:
sales_data_category = sales_data_clean.groupby(["Category"]).aggregate({
    "Unit_Price": 'mean',
    "Discount_Rate": 'mean',
    "Customer_Rating": 'mean',
    "Total_Price": "sum",
    "Discount_Amount": "sum",
    "Net_Sales": "sum",
    "Shipping_Cost": "sum",
    "Profit_Margin": "mean",
    "Profit": "sum",
})
sales_data_category.compute().head()



,Unit_Price,Discount_Rate,Customer_Rating,Total_Price,Discount_Amount,Net_Sales,Shipping_Cost,Profit_Margin,Profit
Category,,,,,,,,,
Books,268.760000,0.098603,3.682796,270046.39,25971.1575,244075.2325,195.93,0.303602,74044.332510
Sports,267.670193,0.102296,3.792271,276001.31,28664.5275,247336.7825,260.58,0.294396,72506.837665
Home & Kitchen,247.157692,0.093478,3.646154,241642.54,23749.6515,217892.8885,279.91,0.294513,63909.347245
Electronics,235.932885,0.097475,3.701923,255943.37,25014.8285,230928.5415,390.73,0.293942,68667.036125
Clothing,253.447304,0.099740,3.794118,261496.17,26038.2680,235457.9020,246.57,0.305980,72432.248095


In [92]:
sales_data_category = sales_data_category.reset_index()
sales_data_category.compute()

,Category,Unit_Price,Discount_Rate,Customer_Rating,Total_Price,Discount_Amount,Net_Sales,Shipping_Cost,Profit_Margin,Profit
0,Books,268.760000,0.098603,3.682796,270046.39,25971.1575,244075.2325,195.93,0.303602,74044.332510
1,Sports,267.670193,0.102296,3.792271,276001.31,28664.5275,247336.7825,260.58,0.294396,72506.837665
2,Home & Kitchen,247.157692,0.093478,3.646154,241642.54,23749.6515,217892.8885,279.91,0.294513,63909.347245
3,Electronics,235.932885,0.097475,3.701923,255943.37,25014.8285,230928.5415,390.73,0.293942,68667.036125
4,Clothing,253.447304,0.099740,3.794118,261496.17,26038.2680,235457.9020,246.57,0.305980,72432.248095


In [65]:
# sales_data_clean = sales_data_clean.drop('index', axis = 1)
# sales_data_clean.compute()

In [107]:
sales_data_clean.compute()

,Order_ID,Date,Category,Region,Customer_Segment,Payment_Method,Quantity,Unit_Price,Discount_Rate,Customer_Rating,...,Discount_Amount,Net_Sales,Shipping_Cost,Profit_Margin,Profit,Month,Day,Only_Month,Year,DayOfWeek
0,ORD-10000,1970-01-01 07:30:45,Books,East,Business,PayPal,1,316.58,0.20,5.0,...,63.3160,253.2640,0.00,0.33,83.577120,2024-07-01,1,7,2024,0
1,ORD-10001,1970-01-01 07:30:46,Sports,West,Premium,Bank Transfer,5,279.76,0.10,4.0,...,139.8800,1258.9200,0.00,0.20,251.784000,2025-03-01,1,3,2025,5
2,ORD-10002,1970-01-01 07:30:46,Home & Kitchen,Central,Business,Bank Transfer,6,209.61,0.05,4.0,...,62.8830,1194.7770,0.00,0.37,442.067490,2024-12-01,1,12,2024,6
3,ORD-10003,1970-01-01 07:30:45,Home & Kitchen,North,Regular,Credit Card,3,265.47,0.15,4.0,...,119.4615,676.9485,0.00,0.23,155.698155,2024-07-01,1,7,2024,0
4,ORD-10004,1970-01-01 07:30:45,Electronics,North,New,Bank Transfer,9,449.32,0.00,4.0,...,0.0000,4043.8800,0.00,0.37,1496.235600,2024-06-01,1,6,2024,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,ORD-10995,1970-01-01 07:30:46,Sports,North,New,PayPal,5,136.47,0.15,4.0,...,102.3525,579.9975,0.00,0.39,226.199025,2024-08-01,1,8,2024,3
996,ORD-10996,1970-01-01 07:30:46,Electronics,Central,Business,Credit Card,3,237.35,0.05,4.0,...,35.6025,676.4475,0.00,0.39,263.814525,2025-02-01,1,2,2025,5
997,ORD-10997,1970-01-01 07:30:45,Books,West,Premium,Cash on Delivery,1,75.31,0.10,2.0,...,7.5310,67.7790,5.16,0.22,14.911380,2024-05-01,1,5,2024,2
998,ORD-10998,1970-01-01 07:30:46,Clothing,South,New,Cash on Delivery,8,464.60,0.10,4.0,...,371.6800,3345.1200,0.00,0.39,1304.596800,2025-01-01,1,1,2025,2


In [122]:
# save to file csv
import os
file_name = '/home/user/pandas-polar-dask/data/_2_silver_data/Silver_Dask_Sale_Data.csv'
if os.path.exists(file_name):
    os.remove(file_name)
    print(f"File '{file_name}' has been deleted.")
else:
    print(f"File '{file_name}' does not exist.")
dd.to_csv(df=sales_data_clean, filename=file_name, compute = True, mode= 'a' )

File '/home/user/pandas-polar-dask/data/_2_silver_data/Silver_Dask_Sale_Data.csv' does not exist.


['/home/user/pandas-polar-dask/data/_2_silver_data/Silver_Dask_Sale_Data.csv']

In [123]:
# save to excel
import pandas as pd
import openpyxl
file_name_pd = '/home/user/pandas-polar-dask/data/_2_silver_data/Silver_Dask_Data.xlsx'
sales_data_clean_pd = pd.DataFrame(sales_data_clean.compute())
sales_data_clean_pd.to_excel(file_name_pd, sheet_name= 'Sales_Data', index=False, engine='openpyxl')

In [ ]:
from dask.distributed import Client

# Start a local cluster
client = Client()
print(client)

### basic aggregation and visualize

In [93]:
sales_data_category = sales_data_category.rename(columns ={
    "Unit_Price": 'Unit_Price_Avg',
    "Discount_Rate": 'Discount_Rate_Avg',
    "Customer_Rating": 'Customer_Rating_Avg',
    "Total_Price": "Total_Price_Sum",
    "Discount_Amount": "Discount_Amount_Sum",
    "Net_Sales": "Net_Sales_Sum",
    "Shipping_Cost": "Shipping_Cost_Sum",
    "Profit_Margin": "Profit_Margin_Avg",
    "Profit": "Profit_Sum"
}).compute()
sales_data_category.head()


,Category,Unit_Price_Avg,Discount_Rate_Avg,Customer_Rating_Avg,Total_Price_Sum,Discount_Amount_Sum,Net_Sales_Sum,Shipping_Cost_Sum,Profit_Margin_Avg,Profit_Sum
0,Books,268.760000,0.098603,3.682796,270046.39,25971.1575,244075.2325,195.93,0.303602,74044.332510
1,Sports,267.670193,0.102296,3.792271,276001.31,28664.5275,247336.7825,260.58,0.294396,72506.837665
2,Home & Kitchen,247.157692,0.093478,3.646154,241642.54,23749.6515,217892.8885,279.91,0.294513,63909.347245
3,Electronics,235.932885,0.097475,3.701923,255943.37,25014.8285,230928.5415,390.73,0.293942,68667.036125
4,Clothing,253.447304,0.099740,3.794118,261496.17,26038.2680,235457.9020,246.57,0.305980,72432.248095


In [94]:
sales_data_category.select_dtypes(include='number').columns

Index(['Unit_Price_Avg', 'Discount_Rate_Avg', 'Customer_Rating_Avg',
       'Total_Price_Sum', 'Discount_Amount_Sum', 'Net_Sales_Sum',
       'Shipping_Cost_Sum', 'Profit_Margin_Avg', 'Profit_Sum'],
      dtype='object')

In [95]:
cols_to_round = sales_data_category.select_dtypes(include='number').columns
sales_data_category[cols_to_round] = sales_data_category[cols_to_round].round(2)
sales_data_category

,Category,Unit_Price_Avg,Discount_Rate_Avg,Customer_Rating_Avg,Total_Price_Sum,Discount_Amount_Sum,Net_Sales_Sum,Shipping_Cost_Sum,Profit_Margin_Avg,Profit_Sum
0,Books,268.76,0.10,3.68,270046.39,25971.16,244075.23,195.93,0.30,74044.33
1,Sports,267.67,0.10,3.79,276001.31,28664.53,247336.78,260.58,0.29,72506.84
2,Home & Kitchen,247.16,0.09,3.65,241642.54,23749.65,217892.89,279.91,0.29,63909.35
3,Electronics,235.93,0.10,3.70,255943.37,25014.83,230928.54,390.73,0.29,68667.04
4,Clothing,253.45,0.10,3.79,261496.17,26038.27,235457.90,246.57,0.31,72432.25


In [96]:
sales_data_category.sort_values(['Unit_Price_Avg'], ignore_index= True, inplace= True,ascending = True)
sales_data_category

,Category,Unit_Price_Avg,Discount_Rate_Avg,Customer_Rating_Avg,Total_Price_Sum,Discount_Amount_Sum,Net_Sales_Sum,Shipping_Cost_Sum,Profit_Margin_Avg,Profit_Sum
0,Electronics,235.93,0.10,3.70,255943.37,25014.83,230928.54,390.73,0.29,68667.04
1,Home & Kitchen,247.16,0.09,3.65,241642.54,23749.65,217892.89,279.91,0.29,63909.35
2,Clothing,253.45,0.10,3.79,261496.17,26038.27,235457.90,246.57,0.31,72432.25
3,Sports,267.67,0.10,3.79,276001.31,28664.53,247336.78,260.58,0.29,72506.84
4,Books,268.76,0.10,3.68,270046.39,25971.16,244075.23,195.93,0.30,74044.33


In [98]:
import altair as alt
import pandas as pd  # Needed because Altair works with pandas DataFrames
sales_data_category_pd = sales_data_category.sort_values(['Unit_Price_Avg'], ignore_index= True,ascending = True)
# Create bar chart with Altair
chart = alt.Chart(sales_data_category_pd).mark_bar().encode(
    x=alt.X('Category:N', title='Category'),
    y=alt.Y('Unit_Price_Avg:Q', title='Unit_Price_Avg'),
    color=alt.Color('Category:N', scale=alt.Scale(scheme='viridis')),  # or "category10", "viridis", etc.
    tooltip=['Category', 'Unit_Price_Avg']
).properties(
    title='Unit_Price_Avg by Category',
    width=500,
    height=300
)

chart.display()  # or chart.show() in some environments
'''
Channel Type Codes:

Code	Meaning	Used For
Q	Quantitative	Numeric data, e.g. sales, prices, age
N	Nominal	Categories (unordered), e.g. product names, cities
O	Ordinal	Ordered categories, e.g. star ratings
T	Temporal	Date/time fields
'''

alt.Chart(...)

'\nChannel Type Codes:\n\nCode\tMeaning\tUsed For\nQ\tQuantitative\tNumeric data, e.g. sales, prices, age\nN\tNominal\tCategories (unordered), e.g. product names, cities\nO\tOrdinal\tOrdered categories, e.g. star ratings\nT\tTemporal\tDate/time fields\n'

In [100]:
import altair as alt
import pandas as pd  # Needed because Altair works with pandas DataFrames
Net_Sales_Sum = sales_data_category.sort_values("Net_Sales_Sum", ascending=True, ignore_index = True)
# Net_Sales_Sum_pd = Net_Sales_Sum.to_pandas()
# Create bar chart with Altair
chart = alt.Chart(Net_Sales_Sum).mark_bar(color='steelblue').encode(
    x=alt.X('Category:N', title='Category', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Net_Sales_Sum:Q', title='Net_Sales_Sum'),
    color=alt.Color('Category:N'),  
    tooltip=['Category', 'Net_Sales_Sum']
).properties(
    title='Net_Sales_Sum by Category',
    width=500,  # 🔹 Set chart width
    height=300
)
# Text labels on top of bars
text = alt.Chart(Net_Sales_Sum).mark_text(
    align='center',
    baseline='bottom',
    dy=-5,  # move text 5px above the bar
    fontSize=12
).encode(
    x='Category:N',
    y='Net_Sales_Sum:Q',
    text='Net_Sales_Sum:Q'
)

# Combine chart and text
chart = chart + text
chart.properties(title='Net_Sales_Sum').configure_axis(labelFontSize=12, titleFontSize=14)
chart.display()

alt.LayerChart(...)

In [101]:
import altair as alt
import pandas as pd  # Needed because Altair works with pandas DataFrames
sales_data_category_pd = sales_data_category.sort_values("Customer_Rating_Avg", ascending=False)
# sales_data_category_pd = sales_data_category_pd.to_pandas()
# Create bar chart with Altair
chart = alt.Chart(sales_data_category_pd).mark_bar().encode(
    x=alt.X('Category:N', title='Category'),
    y=alt.Y('Customer_Rating_Avg:O', title='Customer_Rating_Avg'),
    color=alt.Color('Category:N', scale=alt.Scale(scheme='viridis')),  # or "category10", "viridis", etc.
    tooltip=['Category', 'Customer_Rating_Avg']
).properties(
    title='Customer_Rating_Avg by Category',
    width=500,
    height=300
)

chart.display()

alt.Chart(...)

In [102]:
Net_Sales_Sum_All = Net_Sales_Sum["Net_Sales_Sum"].sum()
Net_Sales_Sum_All

np.float64(1175691.34)

In [104]:
Net_Sales_Sum['Net_Sales_Ratio'] = (Net_Sales_Sum['Net_Sales_Sum'] / Net_Sales_Sum_All * 100).round(2)

Net_Sales_Sum[['Category', 'Net_Sales_Sum', 'Net_Sales_Ratio']]

,Category,Net_Sales_Sum,Net_Sales_Ratio
0,Home & Kitchen,217892.89,18.53
1,Electronics,230928.54,19.64
2,Clothing,235457.90,20.03
3,Books,244075.23,20.76
4,Sports,247336.78,21.04


In [105]:
# Create pie chart
chart = alt.Chart(Net_Sales_Sum).mark_arc(innerRadius=50, tooltip = True).encode(
    theta=alt.Theta(field="Net_Sales_Ratio", type="quantitative"),
    color=alt.Color(field="Category", type="nominal"),
).properties(
    width=400,
    height=400,
    title="Net_Sales_Ratio by Category"
)

# Combine chart and labels
chart.display()

alt.Chart(...)